## Live Angara.Chart 
There is an ability to dynamically update data displayed by Angara.Charting library.
This can be done by returning `AsyncSeq<Chart>` object.

*Note: For now it works only in Jupyter fo Windows*

In [1]:
// To enable the feature we need to have these scripts loaded

// First, paket installations
#load "Angara.Charting.Paket.fsx"
#load "AsyncDisplay.Paket.fsx"

// Then activations
#load "AsyncDisplay.fsx"
#load "Angara.Charting.fsx"
#load "Angara.Charting.Dynamic.fsx"

In [2]:
open System
open FSharp.Control
open Angara.Charting

let phi_0 = 0.0 // initial phase
let N = 100 // number of chart points

let x = Array.init N (fun i -> float(i)/float(N) * 2.0*Math.PI)

/// the generator produces the next chart state using the previous state
/// the state is a phase value
let generator recent_phase =
    async {
        let new_phase = recent_phase + Math.PI/30.0
        
        // generating values
        let y = Array.map (fun x -> Math.Sin(x+new_phase)) x
        
        let chart = 
            [
                Angara.Charting.Plot.markers(x,y)
            ] |> Chart.ofList
        do! Async.Sleep 100 // slowing down the sequence generation process
        return Some (chart,new_phase)
        }
    
let live_chart = //this is infinitly updated chart
    AsyncSeq.unfoldAsync generator phi_0
    
//returning only 200 first iterations
AsyncSeq.take 200 live_chart